In [676]:
#modeling simple hmm

In [1035]:
import numpy as np
import pandas as pd

In [1036]:
def normalizeoveri(dfin):
    for col in dfin.columns:
        tot=sum(dfin.loc[:,col])
        for row in dfin.index:
            dfin.loc[row,col]=dfin.loc[row,col]/tot
    return dfin

In [1037]:
#Alpha (Forward)
def alpha(curseq,states,init,tran,emis):
    seqiter=range(0,len(curseq))
    forward=pd.DataFrame(0,index=states,columns=seqiter)
    for i in seqiter:
        for state in states:
            if i==0:
                forward.loc[state,i]=init[state]*emis.loc[state,curseq[i]]
            else:
                for prevstate in states:
                    forward.loc[state,i]+=forward.loc[prevstate,i-1]*tran.loc[prevstate,state]*emis.loc[state,curseq[i]]
    #display(forward)
    return forward

#for seqtype in corpus:
#    corpus[seqtype]['alpha']=alpha(seqtype,states,init,tran,emis)
#    corpus[seqtype]['prob']=0
#    for state in states:
#        corpus[seqtype]['prob']+=corpus[seqtype]['alpha'].loc[state,corpus[seqtype]['alpha'].shape[1]-1]
#
#corpus

In [1038]:
#Beta (Backward)
def beta(curseq,states,init,trans,emis):
    seqiter=range(len(curseq)-1,-1,-1)
    backward=pd.DataFrame(0,index=states,columns=seqiter)
    for i in seqiter:
        for state in states:
            if i==len(curseq)-1:
                backward.loc[state,i]=1
            else:
                for nextstate in states:
                    backward.loc[state,i]+=backward.loc[nextstate,i+1]*tran.loc[state,nextstate]*emis.loc[nextstate,curseq[i+1]]
    #display(backward)
    return backward
    
#for seqtype in corpus:
#    corpus[seqtype]['beta']=beta(seqtype,states,init,tran,emis)

In [1039]:
#Gamma (Transition)
def gamma(curseq,states,init,tran,emis,alpha,beta,totprob):
    seqiter=range(len(curseq)-1)
    path=pd.DataFrame(0,columns=pd.MultiIndex.from_product([states,states]),index=seqiter) #[state1,state2,time]
    for i in seqiter:
        for state1 in states:
            for state2 in states:
                path.loc[i,(state1,state2)]=alpha.loc[state1,i]*tran.loc[state1,state2]*emis.loc[state2,curseq[i+1]]*beta.loc[state2,i+1]/totprob
    #display(path)
    return(path)
    
#for seqtype in corpus:
#    corpus[seqtype]['gamma']=gamma(seqtype,states,init,tran,emis,corpus[seqtype]['alpha'],corpus[seqtype]['beta'],corpus[seqtype]['prob'])   

In [1040]:
#Delta (State)
def delta(curseq,states,alpha,gamma,totprob):
    seqiter=range(len(curseq))
    single=pd.DataFrame(0,index=states,columns=seqiter)
    for i in seqiter:
        for state in states:
            if i==len(curseq)-1:
                single.loc[state,i]=alpha.loc[state,i]/totprob
            else:
                for nextstate in states:
                    single.loc[state,i]+=gamma.loc[i,(state,nextstate)]
    #display(single)
    return single
    
#for seqtype in corpus:
#    corpus[seqtype]['delta']=delta(seqtype,states,corpus[seqtype]['alpha'],corpus[seqtype]['gamma'],corpus[seqtype]['prob'])

In [1014]:
#Baum-Welch iteration

In [1041]:
#get new init
def getnewinit(corpus,states):
    rawnewinit={}
    for state in states:
        rawnewinit[state]=0
        for seqtype in corpus:
            rawnewinit[state]+=corpus[seqtype]['delta'].loc[state,0]*corpus[seqtype]['num']
    newinit={}
    newtotal=0
    for state in states:
        newtotal+=rawnewinit[state]
    for state in states:
        newinit[state]=rawnewinit[state]/newtotal
    return newinit

#getnewinit(corpus,states)

In [1042]:
#get new tran
def getnewtran(corpus,states):
    newtran=pd.DataFrame(0,columns=states,index=states)
    for state in states:
        rawtran={}
        for nextstate in states:
            rawtran[nextstate]=0
            for seqtype in corpus:
                seqiter=range(0,len(seqtype)-1)
                for i in seqiter:
                    rawtran[nextstate]+=corpus[seqtype]['gamma'].loc[i,(state,nextstate)]*corpus[seqtype]['prob']
        totrawtran=0
        for tempstate in rawtran:
            totrawtran+=rawtran[tempstate]
        for tempstate in rawtran:
            newtran.loc[state,tempstate]=rawtran[tempstate]/totrawtran
    return newtran

#getnewtran(corpus,states)

In [1043]:
#get new emis
def getnewemis(corpus,states,symbols):
    newemis=pd.DataFrame(0,columns=symbols,index=states)
    for state in states:
        rawemis={}
        for symbol in symbols:
            rawemis[symbol]=0
        for seqtype in corpus:
            seqiter=range(0,len(seqtype))
            for i in seqiter:
                rawemis[seqtype[i]]+=corpus[seqtype]['delta'].loc[state,i]*corpus[seqtype]['prob']
        totrawemis=0
        for tempsymbol in rawemis:
            totrawemis+=rawemis[tempsymbol]
        for tempsymbol in rawemis:
            newemis.loc[state,tempsymbol]=rawemis[tempsymbol]/totrawemis
    return newemis

#getnewemis(corpus,states,symbols)

In [1274]:
def loglikelihood(corpus):
    #run alpha to get new prob
    for seqtype in corpus:
        corpus[seqtype]['tempalpha']=alpha(seqtype,states,init,tran,emis)
        corpus[seqtype]['prob']=0
        for state in states:
            corpus[seqtype]['prob']+=corpus[seqtype]['tempalpha'].loc[state,corpus[seqtype]['tempalpha'].shape[1]-1]
    tot=0
    for seqtype in corpus:
        tot+=corpus[seqtype]['num']*np.log(corpus[seqtype]['prob'])
    return tot
#loglikelihood(corpus)

In [1402]:
def baumwelch_iter(corpus,states,symbols,init,tran,emis,accuracy):
    #alpha with terminating accuracy
    diffsum=0
    first=False
    for seqtype in corpus:
        if 'alpha' not in corpus[seqtype]:
            first=True
        else:
            oldalpha=corpus[seqtype]['alpha']
        corpus[seqtype]['alpha']=alpha(seqtype,states,init,tran,emis)
        corpus[seqtype]['prob']=0
        for state in states:
            corpus[seqtype]['prob']+=corpus[seqtype]['alpha'].loc[state,corpus[seqtype]['alpha'].shape[1]-1]
        corpus[seqtype]['alpha']=normalizeoveri(corpus[seqtype]['alpha'])
        if not first:
            diffsum+=abs(oldalpha-corpus[seqtype]['alpha']).sum().sum()
    print('diffsum: '+str(diffsum))
    if not first and diffsum<accuracy:
        return (True,init,tran,emis)
    #beta
    for seqtype in corpus:
        corpus[seqtype]['beta']=normalizeoveri(beta(seqtype,states,init,tran,emis))
    #gamma
    for seqtype in corpus:
        corpus[seqtype]['gamma']=gamma(seqtype,states,init,tran,emis,corpus[seqtype]['alpha'],corpus[seqtype]['beta'],corpus[seqtype]['prob'])
    #delta
    for seqtype in corpus:
        corpus[seqtype]['delta']=delta(seqtype,states,corpus[seqtype]['alpha'],corpus[seqtype]['gamma'],corpus[seqtype]['prob'])
    newinit=getnewinit(corpus,states)
    newtran=getnewtran(corpus,states)
    newemis=getnewemis(corpus,states,symbols)
    #print(loglikelihood(corpus))
    #display(newinit)
    #display(newtran)
    #display(newemis)
    return (False,newinit,newtran,newemis)

In [1330]:
#Data
sequence1=tuple(['B','A','A','A'])
sequence2=tuple(['A','B','B','A'])
corpus={}
corpus[sequence1]={'num':10}
corpus[sequence2]={'num':20}
states=['s','t']
symbols=[]
for seqtype in corpus:
    symbols+=seqtype
symbols=set(symbols)
display(corpus)
#Initial probabilities
init={'s':0.5,'t':0.5}
display(init)
#Transition Matrix
tran=pd.DataFrame([[0.5,0.5],[0.5,0.5]],columns=states,index=states)
display(tran)
#Emission Matrix
emis=pd.DataFrame([[0.3,0.7],[0.7,0.3]],columns=['A','B'],index=states)
display(emis)

{('A', 'B', 'B', 'A'): {'num': 20}, ('B', 'A', 'B'): {'num': 10}}

{'s': 0.5, 't': 0.5}

,s,t
s,0.5,0.5
t,0.5,0.5


,A,B
s,0.3,0.7
t,0.7,0.3


In [1331]:
accuracy=0.01
for i in range(0,30):
    (stop,init,tran,emis)=baumwelch_iter(corpus,states,symbols,init,tran,emis,accuracy)
    if stop:
        break
    print(loglikelihood(corpus))
display(init)
display(tran)
display(emis)

-73.7104429829
-72.2896108012
-71.4229657808
-70.8192980654
-70.211885682
-69.346963223
-67.9059328088
-65.5832133524
-62.4316846194
-59.1453685209
-56.423383266
-54.5688949293
-53.8018567743
-54.1360219821
-55.4449956128
-57.638292855


{'s': 0.38288912109855205, 't': 0.61711087890144789}

,s,t
s,0.208518,7.914823e-01
t,1.000000,1.067983e-27


,B,A
s,9.997287e-01,0.000271
t,3.860760e-12,1.000000


In [1329]:
#for seqtype in corpus:
#    display(seqtype)
#    display(corpus[seqtype]['alpha'])
#    display(corpus[seqtype]['beta'])
#    display(corpus[seqtype]['gamma'])
#    display(corpus[seqtype]['delta'])

In [1429]:
import random
states=['stateA','stateO']
symbols=['A','B','C']
realinit={'stateA':0.5,'stateO':0.5}
realtran=pd.DataFrame([[0.8,0.2],[0.2,0.8]],columns=states,index=states)
realemis=pd.DataFrame([[1.0,0.0,0.0],[0.0,0.5,0.5]],columns=symbols,index=states)
display(realinit)
display(realtran)
display(realemis)
def generatesequences(init,tran,emis,seqnum):
    corpus={}
    for i in range(0,seqnum):
        curseq=[]
        for curpos in range(0,random.randint(30,40)):
            if curpos==0:
                curstate=np.random.choice(a=list(init.keys()),p=list(init.values()))
            else:
                curstate=np.random.choice(a=tran.columns.tolist(),p=tran.loc[curstate,:].tolist())
            curseq.append(np.random.choice(a=emis.columns.tolist(),p=emis.loc[curstate,:].tolist()))
        curseq=tuple(curseq)
        if curseq not in corpus:
            corpus[curseq]={'num':1}
        else:
            corpus[curseq]['num']+=1
    return corpus
corpus=generatesequences(realinit,realtran,realemis,10)
corpus

{'stateA': 0.5, 'stateO': 0.5}

,stateA,stateO
stateA,0.8,0.2
stateO,0.2,0.8


,A,B,C
stateA,1.0,0.0,0.0
stateO,0.0,0.5,0.5


{('A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'B',
  'C',
  'C',
  'B',
  'C',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'C',
  'B',
  'B',
  'C',
  'C',
  'A',
  'C',
  'B',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A'): {'num': 1},
 ('A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'C',
  'C',
  'B',
  'C',
  'B',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'C',
  'B',
  'B',
  'C',
  'C',
  'C',
  'B',
  'C',
  'B',
  'C',
  'A',
  'B',
  'B',
  'B'): {'num': 1},
 ('A',
  'A',
  'B',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'C',
  'B',
  'B',
  'C',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'A',
  'C',
  'A',
  'A',
  'A',
  'A',
  'C',
  'C'): {'num': 1},
 ('A',
  'A',
  'B',
  'A',
  'A',
  'A',
  'C',
  'C',
  'B',
  'A',
  'B',
  'C',
  'B',
  'B',
  'C',
  'B',
  'B',
  'B',
  'A',
  'A',
  'C',
  'C',
  'B',
  'B',
  'B',
  'B',
  'C',
  'B',
  'B',
  'B',
  'B',
  'B',
  'C',
  'A',
  'A',
  'B',
  'B'): {

In [1426]:
init={'stateA':0.8,'stateO':0.2}
tran=pd.DataFrame([[0.4,0.6],[0.6,0.4]],columns=states,index=states)
emis=pd.DataFrame([[0.3,0.4,0.3],[0.4,0.3,0.3]],columns=symbols,index=states)
accuracy=0.1

In [1427]:
for i in range(0,100):
    (stop,init,tran,emis)=baumwelch_iter(corpus,states,symbols,init,tran,emis,accuracy)
    if stop:
        break
    print(loglikelihood(corpus))
display(init)
display(tran)
display(emis)

diffsum: 0
-365.77077389
diffsum: 6.2495716189019745
-371.682585846
diffsum: 8.288972380798931
-373.57329056
diffsum: 9.641657914409373
-373.89030369
diffsum: 9.420903374997124
-373.881394793
diffsum: 9.090877064186992
-373.816057626
diffsum: 8.783746819258447
-373.746648827
diffsum: 8.519531642764182
-373.68335687
diffsum: 8.325868373007472
-373.628340436
diffsum: 8.232249157065265
-373.582299773
diffsum: 8.192157387316355
-373.545856199
diffsum: 8.19367400994121
-373.520007768
diffsum: 8.298089560952212
-373.506519841
diffsum: 8.491978635951467
-373.508488899
diffsum: 8.774660508269891
-373.531124839
diffsum: 9.150955949626423
-373.582275828
diffsum: 9.604382549214035
-373.670807819
diffsum: 10.058942633452881
-373.799614845
diffsum: 10.396897647137976
-373.954034815
diffsum: 10.465079706658502
-374.098466568
diffsum: 10.204411393605264
-374.194859296
diffsum: 9.607390358994294
-374.228682807
diffsum: 8.716801888231416
-374.212750583
diffsum: 7.661150423451441
-374.170112392
diffsum:

{'stateA': 0.88332380074818218, 'stateO': 0.11667619925181792}

,stateA,stateO
stateA,0.237785,0.762215
stateO,0.345676,0.654324


,A,B,C
stateA,0.611270,0.12364,0.265091
stateO,0.688481,0.20414,0.107379


In [1428]:
display(init)
display(tran)
display(emis)

{'stateA': 0.88332380074818218, 'stateO': 0.11667619925181792}

,stateA,stateO
stateA,0.237785,0.762215
stateO,0.345676,0.654324


,A,B,C
stateA,0.611270,0.12364,0.265091
stateO,0.688481,0.20414,0.107379
